In [1]:
import os
import re
import configparser
import shutil

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import  GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, f1_score, confusion_matrix


import mlflow
import pickle

In [2]:
#mlflow ui --backend-store-uri sqlite:///tardy_engine.db

In [3]:
config = configparser.ConfigParser()
config.read('config.ini')
FRAME_SIZE = config.getint('Default', 'FRAME_SIZE')
PATH_PROCESSED_DATA = config.get('Paths', 'PATH_PROCESSED_DATA')
PATH_PROCESSED_SCHEMA = config.get('Paths', 'PATH_PROCESSED_SCHEMA')
DIR_MLRUNS = config.get('Paths', 'DIR_MLRUNS')
experiment_name = "wear_detection_exp"

In [4]:
MLFLOW_TRACKING_URI = "sqlite:///tardy_engine.db"


In [5]:
with open(PATH_PROCESSED_DATA, 'rb') as file:
    print("\t \t Loading processed data =========>>>>>>>>")
    X_train, X_test, Y_train, Y_test = pickle.load(file)
print(X_train.head())

	 	 Loading processed data =========>>>>>>>>
     Mean_b     STD_b  Variance_b     RMS_b  Peak_val_b  Skewness_b  \
0  0.898314  0.184847    3.971968  0.184847    0.034169    0.226163   
1  0.868533  0.291663    3.628897  0.291663    0.085067    0.239338   
2  1.817630  0.315586   15.105156  0.315586    0.099594    0.120332   
3  1.424315  0.228969    5.921020  0.228969    0.052427    0.274177   
4  1.529286  0.412242    7.243120  0.412242    0.169943    0.211136   

   Kurtosis_b  Crest_factor_b  Margin_factor_b  SHape_factor_b  ...     RMS_t  \
0    2.715487        0.528895         7.354278    14942.557270  ...  0.184847   
1   -0.371760        0.598898         3.662079     9636.688906  ...  0.291663   
2    2.397865        0.660802         7.969190    21415.459524  ...  0.315586   
3    7.926048        0.636152        12.531270    27326.475430  ...  0.228969   
4   -0.117123        0.713973         4.587200    14301.089195  ...  0.412242   

   Peak_val_t  Skewness_t  Kurtosis_t  Cr

In [6]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.sklearn.autolog(disable=True)

In [7]:
experiment_name = "Wear Detection"
try:
    experiment_id = mlflow.create_experiment(experiment_name)
except:
    experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id

2024/06/27 13:13:21 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2024/06/27 13:13:21 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

In [8]:
#ML models to train and test
knn_model = KNeighborsClassifier()
rdf_model = RandomForestClassifier()
lgr_model = LogisticRegression()
xgb_model = XGBClassifier()
mlp_model = MLPClassifier()

In [9]:
## Definition of grids
knn_grid = {
    'n_neighbors': [i for i in range(3,33,2)],
    'p':[1, 2],
    'weights' : ["uniform", "distance"]
}

rdf_grid = {
    'n_estimators': [i for i in range(50,550,50)],
    'max_features' : ["sqrt", "log2", None]
}

lgr_grid = {
    'penalty':['l1', 'l2'],
    'solver': ['liblinear', 'saga'],
    'C': [0.1, 1, 10]
}

xgb_grid = {
    'learning_rate' : [0.1, 0.01, 0.001],
    'max_depth' : [i for i in range(3,10,1)],
    'n_estimators': [i for i in range(50,550,50)]
}

mlp_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50), (100, 100), (100,50,20), (100,100,50), (100,100,100), (200,100,50), (200,100,50, 20)],
    'activation': ['relu', 'tanh', 'identity', 'logistic'],
    'solver': ['adam', 'lbfgs', 'sgd']
}
    

In [10]:
gs_knn = GridSearchCV(knn_model, knn_grid, cv=10)
gs_rdf = GridSearchCV(rdf_model, rdf_grid, cv=10)
gs_lgr = GridSearchCV(lgr_model, lgr_grid, cv=10)
gs_xgb = GridSearchCV(xgb_model, xgb_grid, cv=10)
gs_mlp = GridSearchCV(mlp_model, mlp_grid, cv=10)

In [11]:
models_train = {
    "knn": gs_knn,
    "rdf": gs_rdf,
    "lgr": gs_lgr,
    "xgb":gs_xgb,
    "mlp":gs_mlp
}

In [12]:
def model_run(model, X_train, X_test, Y_train, Y_test):
    # Train the model on the train data
    model.fit(X_train, Y_train)
    # Make predictions on the test data
    y_pred = model.best_estimator_.predict(X_test)

    # Calculate standard metrics for classifiers
    metrics = {
        "test_accuracy_score": accuracy_score(Y_test, y_pred),
        "test_f1_score": f1_score(Y_test, y_pred),
        "test_precision_score": precision_score(Y_test, y_pred),
    }
    model_path = "model"
    mlflow.sklearn.log_model(model.best_estimator_, model_path, registered_model_name=f"wear-detection_{run_name}")
    mlflow.log_params(model.best_estimator_.get_params())
    mlflow.log_metrics(metrics)
    mlflow.log_artifact(PATH_PROCESSED_SCHEMA, model_path)

In [13]:
run_id = []
for run_name, mod in models_train.items():
    with mlflow.start_run(run_name=f'{run_name}', experiment_id=experiment_id) as run:
        # Train the model with the current values of hyperparameters, calculate scores, log with mlflow
        model_run(mod, X_train, X_test, Y_train, Y_test)
        run_id.append(run.info.run_id)

/home/codespace/anaconda3/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'wear-detection_knn'.
Created version '1' of model 'wear-detection_knn'.
Successfully registered model 'wear-detection_rdf'.
Created version '1' of model 'wear-detection_rdf'.
/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/codespace/anaconda3/lib/python3.9

/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
Successfully registered model 'wear-detection_lgr'.
Created version '1' of model 'wear-detection_lgr'.
Successfully registered model 'wear-detection_xgb'.
Created version '1' of model 'wear-detection_xgb'.
/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/

/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMIN

/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMIN

/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/utils/extmath.py:153: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/utils/extmath.py:153: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/codespace/anaconda3/lib/python3.9/site-package

...

/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/utils/extmath.py:153: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/utils/extmath.py:153: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/utils/extmath.py:153: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/utils/extmat

/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (

/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (

/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (

Successfully registered model 'wear-detection_mlp'.
Created version '1' of model 'wear-detection_mlp'.


In [14]:
runs = []
for r_id in run_id:
    r_dict = {}
    r = mlflow.get_run(run_id=r_id)
    r_dict["id"] = r_id
    r_dict["test_accuracy_score"] = r.data.metrics["test_accuracy_score"]
    runs.append(r_dict)

In [15]:
best_one = sorted(runs, key=lambda x: x['test_accuracy_score'], reverse = True)[0]
print(best_one)


best_run = mlflow.get_run(run_id=best_one["id"])
target_directory = "../model"

{'id': 'a25d433cfa0c45b19c18fbc9d5c868ce', 'test_accuracy_score': 0.8683127572016461}


In [16]:
# Create the target directory if it doesn't exist
if not os.path.exists(target_directory):
    os.makedirs(target_directory)

# Define the source directory of the model
model_directory = best_run.info.artifact_uri+"/model"
# Copy or move the model files to the target directory
#for root, dirs, files in os.walk(model_directory.split("//")[1]):
#    for file in files:
#        source_path = os.path.join(root, file)
#        target_path = os.path.join(target_directory, file)
#        shutil.copy(source_path, target_path)